# Data analysis

In [ ]:
## For check how the process is being gone through.

In [3]:
from Dataloader_utils import *
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
import cv2
from skimage.feature import hog



In [22]:
def Dataset(train_dataset_path, background_delete=False):
    listdir = os.listdir(train_dataset_path)
    listdir = [listdir for listdir in listdir if not listdir.startswith('.')]  # .DS_Store delete

    train_dataset = []
    valid_dataset = []

    for i, cl_name in enumerate(listdir):
        imgdir = os.listdir(train_dataset_path + cl_name)
        train_num = 0
        for num, img_name in enumerate(imgdir):
            img = cv2.imread(train_dataset_path + cl_name + '/' + img_name, cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if background_delete == True:
                img =  delete_Background(img)

            tmp = [cl_name, img]
            if num < int(len(imgdir) * 0.8):
                train_dataset.append(tmp)
                train_num += 1
            else :
                valid_dataset.append(tmp)
        print('['+str(i+1) +'. '+ cl_name +', [total_data]: ' + str(len(imgdir)) + ' / [train_data]: ' +str(train_num) + ']')
    return train_dataset, valid_dataset



def Dataloder(train_dataset, valid_dataset, option):
    t_labels = []
    v_labels = []
    tmp_list_vec = []
    t_fd_list=[]
    v_fd_list = []
    tmp_list=[]
    if option == 'SIFT':
        #train_dataloder
        for i, (cl_name, img) in enumerate(train_dataset):
            fd = SIFT(img)
            if fd is not None:
                tmp_list.append(fd)
                t_labels.append(cl_name)
        # make it one line
        for descriptor in tmp_list:
            if descriptor is not None:
                for des in descriptor:
                    tmp_list_vec.append(des)

        num_cluster = 60
        BoW = kmean_bow(tmp_list_vec, num_cluster)
        t_fd_list = create_feature_bow(tmp_list, BoW, num_cluster)

        tmp_list=[]
        # valid_dataloder
        for i, (cl_name, img) in enumerate(valid_dataset):
            fd = SIFT(img)
            if fd is not None:
                tmp_list.append(fd)
                v_labels.append(cl_name)
        #use train_dataset BOW
        v_fd_list = create_feature_bow(tmp_list, BoW, num_cluster)


    elif option == 'HOG':
        for i, (cl_name, img) in enumerate(train_dataset):  # 3794 times rotate
            fd = HOG(img)
            if fd is not None:
                t_fd_list.append(fd)
                t_labels.append(cl_name)

        for i, (cl_name, img) in enumerate(valid_dataset):  # 3794 times rotate
            fd = HOG(img)
            if fd is not None:
                v_fd_list.append(fd)
                v_labels.append(cl_name)

    return t_fd_list, t_labels, v_fd_list, v_labels


In [23]:
# Dataload
train_dataset_path = './plant-seedlings-classification/train/'
print('-----traindataset_dataload start-----')
train_dataset, valid_dataset = Dataset(train_dataset_path, True)#train_dataset = (cl_name, img) 3794

-----traindataset_dataload start-----
[1. Cleavers, [total_data]: 287 / [train_data]: 229]
[2. Sugar beet, [total_data]: 385 / [train_data]: 308]
[3. Common Chickweed, [total_data]: 611 / [train_data]: 488]
[4. Loose Silky-bent, [total_data]: 654 / [train_data]: 523]
[5. Scentless Mayweed, [total_data]: 516 / [train_data]: 412]
[6. Shepherds Purse, [total_data]: 231 / [train_data]: 184]
[7. Fat Hen, [total_data]: 475 / [train_data]: 380]
[8. Common wheat, [total_data]: 221 / [train_data]: 176]
[9. Black-grass, [total_data]: 263 / [train_data]: 210]
[10. Small-flowered Cranesbill, [total_data]: 496 / [train_data]: 396]
[11. Charlock, [total_data]: 390 / [train_data]: 312]
[12. Maize, [total_data]: 221 / [train_data]: 176]


# HOG

In [24]:
def Hog(img):
    #let img in gray scale(cv2 img)
    img = cv2.resize(img, dsize=(128, 256), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    fd, hog_image = hog(gray, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True, multichannel=False)
    return fd


In [25]:
# X_train y_train 만들기
t_labels=[]
t_fd_list=[]
v_labels=[]
v_fd_list=[]
for i, (cl_name, img) in enumerate(train_dataset): # 3794 times rotate
    fd = Hog(img)
    if fd is not None:
        t_fd_list.append(fd)
        t_labels.append(cl_name)

for i, (cl_name, img) in enumerate(valid_dataset): # 3794 times rotate
    v_fd = Hog(img)
    if fd is not None:
        v_fd_list.append(v_fd)
        v_labels.append(cl_name)

In [26]:
print(" Training Linear SVM classifier...")
clf_linear = svm.SVC(kernel='rbf',C=6, random_state=42, gamma=0.015)
clf_linear.fit(t_fd_list, t_labels)

 Training Linear SVM classifier...


SVC(C=6, gamma=0.015, random_state=42)

In [27]:
preds = clf_linear.predict(v_fd_list)
acc = accuracy_score(v_labels, preds)
report = classification_report(v_labels, preds)
print(str(round(acc * 100,3)) +'%')
print(report)


67.364%
                           precision    recall  f1-score   support

              Black-grass       0.39      0.32      0.35        53
                 Charlock       0.71      0.87      0.78        78
                 Cleavers       0.65      0.64      0.64        58
         Common Chickweed       0.65      0.76      0.70       123
             Common wheat       0.54      0.29      0.38        45
                  Fat Hen       0.68      0.66      0.67        95
         Loose Silky-bent       0.70      0.80      0.75       131
                    Maize       0.63      0.53      0.58        45
        Scentless Mayweed       0.67      0.69      0.68       104
          Shepherds Purse       0.53      0.40      0.46        47
Small-flowered Cranesbill       0.84      0.86      0.85       100
               Sugar beet       0.72      0.60      0.65        77

                 accuracy                           0.67       956
                macro avg       0.64      0.62      

In [28]:
def SIFT(img):
    img = cv2.resize(img, dsize=(128, 256), interpolation=cv2.INTER_CUBIC)
    sift = cv2.xfeatures2d.SIFT_create()
    _, des = sift.detectAndCompute(img, None)

    return des

In [29]:
t_des_list=[]
t_labels=[]
v_des_list=[]
v_labels=[]
for i, (cl_name, img) in enumerate(train_dataset):
    des=SIFT(img)
    if des is not None:
        t_des_list.append(des)
        t_labels.append(cl_name)

for i, (cl_name, img) in enumerate(valid_dataset):
    des=SIFT(img)
    if des is not None:
        v_des_list.append(des)
        v_labels.append(cl_name)

In [30]:
def kmean_bow(all_descriptors, num_cluster):
    bow_dict = []

    kmeans = KMeans(n_clusters = num_cluster, random_state=42)
    kmeans.fit(all_descriptors)

    bow_dict = kmeans.cluster_centers_

    return bow_dict

In [31]:
t_descriptors = []
v_descriptors = []
for descriptor in t_des_list:
    if descriptor is not None:
        for des in descriptor:
            t_descriptors.append(des)

for descriptor in v_des_list:
    if descriptor is not None:
        for des in descriptor:
            v_descriptors.append(des)

In [32]:
num_cluster=60
t_BoW = kmean_bow(t_descriptors, num_cluster)
X_train = create_feature_bow(t_des_list, t_BoW, num_cluster)

X_valid = create_feature_bow(v_des_list, t_BoW, num_cluster)

In [33]:

clf_linear = svm.SVC(kernel='rbf',C= 1, random_state=42, gamma=0.008)
clf_linear.fit(X_train, t_labels)

preds = clf_linear.predict(X_valid)
acc = accuracy_score(v_labels, preds)
report = classification_report(v_labels, preds)
print(f'{str(round(acc * 100,3))}%')
print(report)

63.246%
                           precision    recall  f1-score   support

              Black-grass       0.58      0.13      0.22        53
                 Charlock       0.59      0.62      0.60        78
                 Cleavers       0.52      0.29      0.37        58
         Common Chickweed       0.51      0.64      0.57       123
             Common wheat       0.72      0.47      0.57        45
                  Fat Hen       0.70      0.77      0.73        95
         Loose Silky-bent       0.71      0.87      0.78       130
                    Maize       0.50      0.18      0.26        45
        Scentless Mayweed       0.63      0.79      0.70       104
          Shepherds Purse       0.62      0.34      0.44        47
Small-flowered Cranesbill       0.80      0.87      0.83       100
               Sugar beet       0.54      0.69      0.60        77

                 accuracy                           0.63       955
                macro avg       0.62      0.55      